In [11]:
from pyspark.sql import SparkSession

In [12]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "C:/Spark/spark-3.4.0-bin-hadoop3"

In [13]:
import findspark
findspark.init()

In [14]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()

In [23]:
df_tweet = spark.read.json("../../datalake/Silver/twitter_datascience/tweet/process_date=2022-11-23/part-00000-4ec0a1be-06fc-4ef3-92aa-625dce46de6b-c000.json")

In [18]:
from pyspark.sql import functions as f

In [19]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [20]:
tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))\
        .agg(
            f.countDistinct("author_id").alias("n_tweets"),
            f.sum("like_count").alias("n_like"),
            f.sum("quote_count").alias("n_quote"),
            f.sum("reply_count").alias("n_reply"),
            f.sum("retweet_count").alias("n_retweet")
        ).withColumn("weekday", f.date_format("created_date", "E"))

In [21]:
tweet_conversas.orderBy("created_date").show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2022-11-23|      42|   152|      1|      6|     2554|    Wed|
+------------+--------+------+-------+-------+---------+-------+

